<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Enusre Long Running AWS ElastiCache Clusters have Reserved Cache Nodes purchased for them.</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Purchase-Reserved-Cache-Nodes-For-Long-Running-AWS-ElastiCache-Clusters"><u>Purchase Reserved Cache Nodes For Long Running AWS ElastiCache Clusters</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find Long Running AWS ElastiCache Clusters without Reserved Cache Nodes</a><br>2)<a href="#2" target="_self" rel="noopener"> Purchase Reserved Cache Nodes</a></p>

In [ ]:
if region == None:
    region = ''
threshold_int = int(threshold)
if reserved_cache_node_offering_id and not region:
    raise SystemExit("Provide a region for the Reserved Cache Node Offering ID!")

<h3 id="Find-ECS-Clusters-with-Low-CPU-Utilization"><a id="1" target="_self" rel="nofollow"></a>Find Long Running AWS ElastiCache Clusters without Reserved Nodes</h3>
<p>Using unSkript's Find Long Running AWS ElastiCache Clusters without Reserved Nodes action, we will find clusters that have been running for longer than a specified threshold and do not have reserved cache nodes purchased for them.</p>
<blockquote>
<p>This action takes the following parameters: <code>region, threshold</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>clusters_without_reserved_nodes</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint
from datetime import datetime,timedelta, timezone



from typing import Optional

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_get_long_running_elasticcache_clusters_without_reserved_nodes_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_long_running_elasticcache_clusters_without_reserved_nodes(handle, region: str = "", threshold:int = 10) -> Tuple:
    """aws_get_long_running_elasticcache_clusters_without_reserved_nodes finds ElasticCache Clusters that are long running and have no reserved nodes

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the Cluster.

        :type threshold: integer
        :param threshold: Threshold(in days) to find long running ElasticCache clusters. Eg: 30, This will find all the clusters that have been created a month ago.

        :rtype: status, list of clusters, nodetype and their region.
    """
    result = []
    reservedNodesPerRegion = {}
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    # Get the list of reserved node per region per type. We just need to maintain
    # what type of reserved nodes are present per region. So, reservedNodesPerRegion
    # would be like:
    # <region>:{<nodeType>:True/False}
    for reg in all_regions:
        try:
            elasticacheClient = handle.client('elasticache', region_name=reg)
            response = elasticacheClient.describe_reserved_cache_nodes()
            reservedNodesPerType = {}
            if response['ReservedCacheNodes']:
                for node in response['ReservedCacheNodes']:
                    reservedNodesPerType[node['CacheNodeType']] = True
            else:
                continue
            reservedNodesPerRegion[reg] = reservedNodesPerType
        except Exception:
            pass

    for reg in all_regions:
        try:
            elasticacheClient = handle.client('elasticache', region_name=reg)
            for cluster in elasticacheClient.describe_cache_clusters()['CacheClusters']:
                cluster_age = datetime.now(timezone.utc) - cluster['CacheClusterCreateTime']
                if cluster_age > timedelta(days=threshold):
                    # Check if the cluster node type is present in the reservedNodesPerRegion map.
                    reservedNodes = reservedNodesPerRegion.get(reg)
                    if reservedNodes != None:
                        if reservedNodes.get(cluster['CacheNodeType']) == True:
                            continue
                    cluster_dict = {}
                    cluster_dict["region"] = reg
                    cluster_dict["cluster"] = cluster['CacheClusterId']
                    cluster_dict["node_type"] = cluster['CacheNodeType']
                    result.append(cluster_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "threshold": "int(threshold_int)"
    }''')
task.configure(outputName="clusters_without_reserved_nodes")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_long_running_elasticcache_clusters_without_reserved_nodes, lego_printer=aws_get_long_running_elasticcache_clusters_without_reserved_nodes_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Clusters-without-reserved-nodes">Create List of Clusters without reserved nodes</h3>
<p>This action filters regions that have no clusters and creates a list of those that have them (without reserved cache nodes).</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_clusters_without_reserved_nodes</code></p>
</blockquote>

In [ ]:
all_clusters_without_reserved_nodes = []
dummy = []
for res in clusters_without_reserved_nodes:
    if type(res)==bool:
        if res == False:
            continue
    elif type(res)==list:
        if len(res)!=0:
            all_clusters_without_reserved_nodes=res
print(all_clusters_without_reserved_nodes)

<h3 id="Purchase-Reserved-Cache-Node"><a id="2" target="_self" rel="nofollow"></a>Purchase Reserved Cache Node</h3>
<p>This action Purchases Reserved Cache Nodes for the clusters found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>no_of_nodes, region, reserved_node_offering_id</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint


from beartype import beartype
@beartype
def aws_purchase_elasticcache_reserved_node_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_purchase_elasticcache_reserved_node(handle, region: str, reserved_node_offering_id: str, no_of_nodes:int=1) -> Dict:
    """aws_purchase_elasticcache_reserved_node returns dict of response.

        :type region: string
        :param region: AWS Region.

        :type reserved_node_offering_id: string
        :param reserved_node_offering_id: The unique identifier of the reserved node offering you want to purchase. Example: '438012d3-4052-4cc7-b2e3-8d3372e0e706'

        :type no_of_nodes: int
        :param no_of_nodes: The number of reserved nodes that you want to purchase.

        :rtype: dict of response metatdata of purchasing a reserved node
    """
    try:
        elasticClient = handle.client('elasticache', region_name=region)
        params = {
            'ReservedCacheNodesOfferingId': reserved_node_offering_id,
            'CacheNodeCount': no_of_nodes
            }
        response = elasticClient.purchase_reserved_cache_nodes_offering(**params)
        return response
    except Exception as e:
        raise Exception(e)




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "reserved_node_offering_id": "reserved_cache_node_offering_id",
    "no_of_nodes": int(no_of_nodes)
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_purchase_elasticcache_reserved_node, lego_printer=aws_purchase_elasticcache_reserved_node_printer, hdl=hdl, args=args)

<h3 id="Conclusion&para;">Conclusion</h3>
<p>In this Runbook, we were able to filter long running ElastiCache clusters without reserved nodes given a threshold number of days of creation and purchase nodes for them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>